In [1]:
with h5py.File('test_cite_inputs_day_2_donor_27678.h5', 'r+') as hdf:
    base_items = list(hdf.items())
    print('items in the base_items directory:', base_items )
    G1 = hdf.get('train_multi_inputs')
    G1_items = list(G1.items())
    print('items in G1:', G1_items)
    block0_values = G1.get('block0_values')
    print('items in block0_values', block0_values)
    print(type(block0_values))

"with h5py.File('test_cite_inputs_day_2_donor_27678.h5', 'r+') as hdf:\n    base_items = list(hdf.items())\n    print('items in the base_items directory:', base_items )\n    G1 = hdf.get('train_multi_inputs')\n    G1_items = list(G1.items())\n    print('items in G1:', G1_items)\n    block0_values = G1.get('block0_values')\n    print('items in block0_values', block0_values)\n    print(type(block0_values))"

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import h5py
import time

In [5]:
data = h5py.File('test_cite_inputs_day_2_donor_27678.h5', 'r+')

In [6]:
data.keys()

<KeysViewHDF5 ['test_cite_inputs_day_2_donor_27678']>

In [7]:
meta_df = pd.read_csv('metadata.csv', index_col = 0)
display(meta_df) 

,day,donor,cell_type,technology
cell_id,,,,
c2150f55becb,2,27678,HSC,citeseq
65b7edf8a4da,2,27678,HSC,citeseq
c1b26cb1057b,2,27678,EryP,citeseq
917168fa6f83,2,27678,NeuP,citeseq
2b29feeca86d,2,27678,EryP,citeseq
...,...,...,...,...
96a60b026659,10,31800,hidden,multiome
d493e546991e,10,31800,hidden,multiome
05666c99aa48,10,31800,hidden,multiome


In [8]:
#import multiprocessing as mp
#print('number of cores:', mp.cpu_count())

In [9]:
X = pd.read_hdf('train_multi_targets.h5') 
display(X)

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,4.893861,0.0,0.0,0.000000,0.0,5.583255,0.000000,4.893861
fc0c60183c33,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9b4a87e22ad0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.107832,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.107832
81cccad8cd81,0.0,4.507936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,5.195558,4.507936,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.195558
15cb3d85c232,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,5.531572,0.0,0.000000,4.842377,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
063cead1a4ea,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,5.829234,0.0,0.000000,0.000000,5.139023
553bca99ba78,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.091473,0.000000,5.091473,0.0,0.0,5.091473,0.0,0.000000,5.091473,0.000000
00783f28b463,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,4.500950,0.0,0.000000,0.000000,0.000000


In [10]:
meta_df['sex'] = np.where(meta_df['donor'] > 30000, 'Male', 'Female')  #(1)
y = meta_df[meta_df.index.isin(X.index)].set_index(X.index)[['sex']]   #(2)
y = pd.get_dummies(y)
print(y, type(y))

              sex_Female  sex_Male
cell_id                           
56390cf1b95e           0         1
fc0c60183c33           0         1
9b4a87e22ad0           0         1
81cccad8cd81           0         1
15cb3d85c232           0         1
...                  ...       ...
063cead1a4ea           0         1
553bca99ba78           0         1
00783f28b463           0         1
e7abb1a0f251           0         1
193992d571a5           0         1

[105942 rows x 2 columns] <class 'pandas.core.frame.DataFrame'>


In [11]:
'''(1) 
In this line, you are adding a new column named 'sex' to the meta_df DataFrame. 
The values in the 'sex' column are assigned based on a condition: if the value in the 'donor' column is 
greater than 30000, it assigns 'Male', otherwise 'Female'. 
This line essentially creates a binary gender classification based on the 'donor' column.'''

"(1) \nIn this line, you are adding a new column named 'sex' to the meta_df DataFrame. \nThe values in the 'sex' column are assigned based on a condition: if the value in the 'donor' column is \ngreater than 30000, it assigns 'Male', otherwise 'Female'. \nThis line essentially creates a binary gender classification based on the 'donor' column."

In [12]:
'''#(2)
This line filters the meta_df DataFrame based on the indices that are present in the 'X' DataFrame. It retrieves 
the rows from meta_df whose indices match with 'X' and selects only the 'sex' column. The resulting DataFrame 
is assigned to the 'y' variable.'''

"#(2)\nThis line filters the meta_df DataFrame based on the indices that are present in the 'X' DataFrame. It retrieves \nthe rows from meta_df whose indices match with 'X' and selects only the 'sex' column. The resulting DataFrame \nis assigned to the 'y' variable."

In [13]:
'''#(3)
This line applies one-hot encoding to the 'y' DataFrame using the pd.get_dummies() function. It converts the 
categorical variable 'sex' into binary columns, where each unique category becomes a separate column with binary 
values (0 or 1) indicating the presence or absence of that category.'''

"#(3)\nThis line applies one-hot encoding to the 'y' DataFrame using the pd.get_dummies() function. It converts the \ncategorical variable 'sex' into binary columns, where each unique category becomes a separate column with binary \nvalues (0 or 1) indicating the presence or absence of that category."

In [14]:
rna_index = X.columns
rna_index

Index(['ENSG00000121410', 'ENSG00000268895', 'ENSG00000175899',
       'ENSG00000245105', 'ENSG00000166535', 'ENSG00000256661',
       'ENSG00000184389', 'ENSG00000128274', 'ENSG00000094914',
       'ENSG00000081760',
       ...
       'ENSG00000086827', 'ENSG00000174442', 'ENSG00000122952',
       'ENSG00000198205', 'ENSG00000198455', 'ENSG00000070476',
       'ENSG00000203995', 'ENSG00000162378', 'ENSG00000159840',
       'ENSG00000074755'],
      dtype='object', name='gene_id', length=23418)

In [15]:
y = y['sex_Male'].values
print(y.shape, type(y))

(105942,) <class 'numpy.ndarray'>


In [16]:
z = y['sex_Female'].values
print(z.shape, type(y))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
X = X.values
print(X.shape, type(X))

(105942, 23418) <class 'numpy.ndarray'>


In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X.shape)

(105942, 23418)


In [19]:
import gc
gc.collect()

499

In [20]:
'''In the code snippet provided, gc.collect() is called after importing the gc module. Here's an explanation of what 
this code does:

gc stands for the garbage collector module in Python. The garbage collector is responsible for automatically 
reclaiming memory that is no longer in use by the program, freeing up resources and preventing memory leaks.
By calling gc.collect(), you are explicitly triggering a garbage collection cycle. This forces the garbage 
collector to run and attempt to reclaim memory that is no longer needed.

The purpose of explicitly calling gc.collect() is to request an immediate garbage collection cycle and potentially 
free up memory that may have been allocated by previous operations or variables that are no longer in use.

It's worth noting that in most cases, Python's garbage collector manages memory automatically and efficiently 
without the need for manual intervention. However, in certain situations, explicitly calling gc.collect() can be 
useful, especially when dealing with large datasets or when you want to ensure timely release of memory.

It's important to use gc.collect() judiciously and only when necessary. Unnecessary or frequent calls to gc.collect() 
may introduce performance overhead. In general, it is recommended to let the garbage collector handle memory 
management automatically, unless you have specific memory-related concerns or requirements.'''

"In the code snippet provided, gc.collect() is called after importing the gc module. Here's an explanation of what \nthis code does:\n\ngc stands for the garbage collector module in Python. The garbage collector is responsible for automatically \nreclaiming memory that is no longer in use by the program, freeing up resources and preventing memory leaks.\nBy calling gc.collect(), you are explicitly triggering a garbage collection cycle. This forces the garbage \ncollector to run and attempt to reclaim memory that is no longer needed.\n\nThe purpose of explicitly calling gc.collect() is to request an immediate garbage collection cycle and potentially \nfree up memory that may have been allocated by previous operations or variables that are no longer in use.\n\nIt's worth noting that in most cases, Python's garbage collector manages memory automatically and efficiently \nwithout the need for manual intervention. However, in certain situations, explicitly calling gc.collect() can be \nusef

In [21]:
train_size = 0.7
print('X.shape, y.shape:', X.shape, y.shape, 'train_size:', train_size)


p = np.random.permutation(len(y))                                     #1
N = int(len(y) *  train_size)                                         #2
IX_train = np.arange(len(y))[p][:N]                                   #3
IX_test =  np.arange(len(y))[p][N:] 
print(" len(IX_train), len(IX_test):",  len(IX_train), len(IX_test) ) #4

X.shape, y.shape: (105942, 23418) (105942,) train_size: 0.7
 len(IX_train), len(IX_test): 74159 31783


In [22]:
''' 

#1 This line generates a random permutation of integers from 0 to the length of the array y. The permutation is 
created using the np.random.permutation() function from the NumPy library. This random permutation is typically 
used for shuffling the indices of the data to create a randomized order.

#2 This line calculates the number of samples to be included in the training set. It multiplies the length of the 
array y by the train_size (variable) (which is set to 0.7 in this case) and converts the result to an integer using 
the int() function. The value N represents the number of samples to be included in the training set based on the 
specified training size.

#3 This line creates an array of indices for the training set. It uses np.arange(len(y)) to generate an array of 
indices from 0 to the length of y, and then it applies the random permutation p to shuffle the indices. Finally, 
it selects the first N indices using the slicing notation [:N], representing the indices for the training set.

#4 This line prints the lengths of the IX_train and IX_test arrays, representing the number of samples in the 
training and test sets, respectively.

Overall, this code snippet generates a random permutation of indices, splits the data into training and test sets 
based on the specified train_size, and prints the lengths of the resulting sets. The IX_train and IX_test arrays 
can be used to index the X and y data to obtain the corresponding training and test data subsets.


'''

' \n\n#1 This line generates a random permutation of integers from 0 to the length of the array y. The permutation is \ncreated using the np.random.permutation() function from the NumPy library. This random permutation is typically \nused for shuffling the indices of the data to create a randomized order.\n\n#2 This line calculates the number of samples to be included in the training set. It multiplies the length of the \narray y by the train_size (variable) (which is set to 0.7 in this case) and converts the result to an integer using \nthe int() function. The value N represents the number of samples to be included in the training set based on the \nspecified training size.\n\n#3 This line creates an array of indices for the training set. It uses np.arange(len(y)) to generate an array of \nindices from 0 to the length of y, and then it applies the random permutation p to shuffle the indices. Finally, \nit selects the first N indices using the slicing notation [:N], representing the in

In [ ]:
log_reg = LogisticRegression(penalty='l1', C = 0.1, solver = 'saga', max_iter=500) #1
log_reg.fit(X[IX_train,:], y[IX_train])                                            #2

y_pred = log_reg.predict(X[IX_test])                                               #3

auc_score = roc_auc_score(y[IX_test], y_pred)                                      #4

print("AUC Score: ", auc_score)                                                    #5
weights_dict = {}                                                                  #6
weights_dict['coef'] = log_reg.coef_                                               #7
print('non-zero coeffs:', (log_reg.coef_ != 0 ).sum())                             #8

In [ ]:
'''
#1 This line creates an instance of the logistic regression model using the LogisticRegression class from 
scikit-learn. The model is initialized with the specified parameters:

. penalty='l1' sets the penalty to L1 regularization, which promotes sparsity in the model's coefficients.
. C=0.1 is the inverse of the regularization strength. Smaller values of C increase the regularization, 
  helping to prevent overfitting.
. solver='saga' selects the 'saga' solver, which is suitable for L1 penalty and supports both binary and 
  multiclass problems.
. max_iter=500 sets the maximum number of iterations for the solver.

#2 This line fits the logistic regression model to the training data. The fit() method takes the features X 
indexed by IX_train and the corresponding labels y indexed by IX_train. It trains the model by finding the optimal 
coefficients that minimize the logistic loss function.

#3 This line predicts the labels for the test data using the trained logistic regression model. The predict() 
method takes the features X indexed by IX_test and returns the predicted labels.

#4 This line calculates the area under the receiver operating characteristic curve (AUC) as a performance metric
for the model. The roc_auc_score() function from scikit-learn is used, taking the true labels y indexed by IX_test 
and the predicted labels y_pred as inputs.

#5 This line prints the AUC score, representing the performance of the logistic regression model on the test data.

#6 This code creates a dictionary weights_dict and assigns the coefficients of the logistic regression model to 
the 'coef' key in the dictionary. The coefficients represent the weights assigned to each feature in the logistic
regression model.

#7 This line prints the number of non-zero coefficients in the logistic regression model. It calculates the sum of 
boolean values (log_reg.coef_ != 0) which evaluates to True for non-zero coefficients and False for zero coefficients. The resulting sum represents the count of non-zero coefficients.

Overall, this code trains a logistic regression model, predicts labels for the test data, evaluates the model's performance using the AUC score, and prints the AUC score as well as
the number of non-zero coefficients in the model.

'''